In [ ]:
import pandas as pd
from datetime import timedelta,datetime
import clr

d = GDFL_1MinData.GDFL_1MIN_OHLC()
excel = pd.DataFrame(pd.read_csv('BankNiftyATM_call.csv'))

clr.FindAssembly('BlackScholesCSharpeDLL.dll')
clr.AddReference('BlackScholesCSharpeDLL')
from BlackScholesCSharpeDLL import BSCalculator

excel['Timestamp']=pd.to_datetime(excel['Timestamp'], format='%Y-%m-%d %H:%M:%S')
excel.set_index(["DATE"],inplace=True)

allDates =list(pd.Series(excel.index).unique())

rf=0.06/(365*24*60)

for date in allDates[:]:

    exception= datetime(2019,5,30)

    if (pd.to_datetime(date)).date()!=exception.date():
        dayTrades = excel.loc[[date]]

        if len(dayTrades)!=1:
            time1 = (dayTrades['Timestamp'].iloc[0])
            time2 = dayTrades['Timestamp'].iloc[1]

            entryTime = min(time1,time2)
            exitTime = max(time1,time2)

        else:
            continue
            entryTime= dayTrades['Timestamp'].iloc[0]
            exitTime =dayTrades['Timestamp'].iloc[0]+timedelta(seconds=21900)

        dayTrades.set_index(['Timestamp'],inplace=True)

        qty = abs(dayTrades.loc[entryTime,'OrderQuantity'])

        side = dayTrades.loc[entryTime,'OrderSide']
        if side=="BUY":
            holdings = qty
        else:
            holdings = -qty

        type1 = dayTrades['OptionType'].iloc[0]
        if type1=='CE':
            type2='c'
        else:
            type2='p'

        strike = dayTrades['Strike_Pr'].iloc[0]

        expirydate = pd.to_datetime(dayTrades['ExpiryDate'].iloc[0])
        expirytimestamp=pd.to_datetime(dayTrades['ExpiryDate'].iloc[0]+timedelta(seconds=55799))

        entryTime=entryTime.strftime('%Y-%m-%d %H:%M')+':59'
        exitTime=exitTime.strftime('%Y-%m-%d %H:%M')+':59'

        entryTime=pd.to_datetime(entryTime)
        exitTime=pd.to_datetime(exitTime)

        optionData=d.read_options_data('BANKNIFTY', strike, type1, entryTime.date(), expirytimestamp.date())
        optionData.set_index("DateTime",inplace=True)
        optionData= optionData.loc[entryTime:exitTime]

        indexData=d.read_indices_data('NIFTY BANK', entryTime.date())
        indexData.set_index("DateTime",inplace=True)
        indexData= indexData.loc[entryTime:exitTime]

        greeks = pd.DataFrame(columns=["DATETIME","IV","DELTA","GAMMA","VEGA","THETA","VANNA","VOMMA","SPOTPRICE","HOLDINGS","PREMIUM","TIME_TO_EXIRY"])

        for datetime1 in optionData.index:
            if datetime1 in indexData.index:

                timetoexpiry=((expirytimestamp-datetime1).total_seconds())/60

                spot=indexData.Close.loc[datetime1]
                premium=optionData.Close.loc[datetime1]

                if entryTime<datetime1 and datetime1<=exitTime:
                    holdings=qty
                else:
                    holdings=0

                vol=BSCalculator.BSIVBinarySearch(type2,spot, strike,rf,timetoexpiry,premium,0,3,0.00001,1)

                delta=BSCalculator.BSDelta(type2,spot,strike,rf,timetoexpiry,vol)
                gamma=BSCalculator.BSGamma(type2,spot,strike,rf,timetoexpiry,vol)
                vega=BSCalculator.BSVega(type2,spot,strike,rf,timetoexpiry,vol)
                theta=BSCalculator.BSTheta(type2,spot,strike,rf,timetoexpiry,vol)
                vanna=BSCalculator.BSVanna(type2,spot,strike,rf,timetoexpiry,vol)
                vomma=BSCalculator.BSVomma(type2,spot,strike,rf,timetoexpiry,vol)

                try:
                    theta>0
                except:
                    print("error")
                greeks_add=pd.DataFrame([[datetime1,vol, delta,gamma,vega,theta,vanna,vomma,spot,holdings,premium,timetoexpiry]],columns=["DATETIME","IV","DELTA","GAMMA","VEGA","THETA","VANNA","VOMMA","SPOTPRICE","HOLDINGS","PREMIUM","TIME_TO_EXIRY"])
                greeks = greeks.append(greeks_add,ignore_index=False)
        greeks.ffill(axis = 0,inplace=True)
        greeks.set_index(['DATETIME'],inplace=True)
        greeks1=greeks.shift(1,axis=0)

        for datetime2 in greeks.index[1:]:

            changespot=greeks.SPOTPRICE.loc[datetime2]-greeks1.SPOTPRICE.loc[datetime2]
            changeIV=greeks.IV.loc[datetime2]-greeks1.IV.loc[datetime2]
            changepremium=greeks.PREMIUM.loc[datetime2]-greeks1.PREMIUM.loc[datetime2]
            changetimetoexpiry=greeks1.TIME_TO_EXIRY.loc[datetime2]-greeks.TIME_TO_EXIRY.loc[datetime2]

            holdings=greeks.HOLDINGS.loc[datetime2]

greeks.set_index(['DATE'],inplace=True)
print(greeks)
greeks.to_excel('GREEKS.xlsx')